# Importações

In [ ]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
import pandas as pd

# Aquisição de dados

In [2]:
from RainPredictionMachine.data import clean_data_rpm
df = clean_data_rpm()
df = df.clean_data(1)
df

,Chuva,Pres,Pres_max,Pres_min,Radiacao,Temp,Temp_orvalho,Temp_max,Temp_min,Temp_orvalho_max,...,Umid,Dir_vento,Rajada_vento,Vel_vento,Estaçao,Latitude,Longitude,Altitude,datahora,classe_chuva
0,0.0,957.9,957.9,957.1,0.0,21.7,21.7,23.7,21.7,23.7,...,100.0,130.5,3.75,2.10,TUPA,-21.927251,-50.490251,498.0,2021-01-01 00:00:00+00:00,nao chove
1,0.0,958.0,958.3,957.9,0.0,21.4,21.4,21.7,21.4,21.6,...,100.0,109.0,5.40,3.35,TUPA,-21.927251,-50.490251,498.0,2021-01-01 01:00:00+00:00,nao chove
2,0.0,957.4,958.0,957.4,0.0,21.2,21.2,21.5,21.2,21.5,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 02:00:00+00:00,nao chove
3,0.0,956.6,957.4,956.6,0.0,21.0,21.0,21.3,20.9,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 03:00:00+00:00,nao chove
4,0.0,957.0,957.0,956.6,0.0,21.3,21.3,21.3,21.0,21.3,...,100.0,160.5,3.20,1.40,TUPA,-21.927251,-50.490251,498.0,2021-01-01 04:00:00+00:00,nao chove
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,0.0,947.7,948.2,947.6,2930.5,30.0,18.7,30.4,28.6,20.0,...,51.0,238.0,8.40,2.70,TUPA,-21.927251,-50.490251,498.0,2021-12-31 19:00:00+00:00,nao chove
8756,0.0,947.6,947.7,947.5,1773.5,29.6,17.9,30.5,29.4,19.1,...,50.0,202.0,6.40,2.00,TUPA,-21.927251,-50.490251,498.0,2021-12-31 20:00:00+00:00,nao chove
8757,0.0,948.0,948.0,947.6,1157.7,25.7,20.7,29.6,25.7,21.0,...,74.0,19.0,6.20,2.90,TUPA,-21.927251,-50.490251,498.0,2021-12-31 21:00:00+00:00,nao chove
8758,0.0,948.4,948.4,948.0,219.7,24.7,19.9,26.0,24.7,21.4,...,75.0,48.0,4.60,1.40,TUPA,-21.927251,-50.490251,498.0,2021-12-31 22:00:00+00:00,nao chove


# Processamento

In [ ]:
from RainPredictionMachine.Trainer import pipe_creator
X_train_t, X_test_t  = pipe_creator(df)

In [1]:
X_train_t.shape

NameError: name 'X_train_t' is not defined

# construção da RNN

## normalização

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train)
normalizer(X_train)[0]

## camadas da RNN

In [ ]:
model = Sequential()
model.add(normalizer)
model.add(SimpleRNN(units=20, activation='tanh'))
model.add(Dense(10, activation='relu'))
model.add(Dense(5, activation='softmax'))

## compilando

In [ ]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    return model

model = compile_model(model)

# Ajuste da RNN

Fazendo um early stopping

In [ ]:
from keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

Ajustando o modelo

In [ ]:
model.fit(X_train, y_train,
          epochs=100, 
          batch_size=32, 
          validation_split=0.2,
          callbacks=[es])

In [ ]:
df


In [ ]:
df['classe_chuva'].shift(-1)